# Creating the readme for the unprocessed documents list

In [41]:
import sys

!{sys.executable} -m pip install tabulate

  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip


In [49]:
import sys

sys.path.append("..")
from pathlib import Path

import os

import pandas as pd
from IPython.display import Markdown

## 1. Get metadata of documents that couldn't be parsed

In [72]:
# Get IDs of missing documents
missing_docs_df = pd.read_excel("../data/raw/No Text Documents.xlsx")
missing_docs_ids = missing_docs_df["Document ID"].tolist()

extra_ids = [
    "UNFCCC.party.178.0",
    "UNFCCC.party.228.0",
    "UNFCCC.party.134.0",
    "UNFCCC.party.177.0",
    "UNFCCC.party.809.0",
    "UNFCCC.party.811.0",
    "UNFCCC.non-party.288.0",
    "UNFCCC.party.389.0",
    "UNFCCC.party.136.0",
    "UNFCCC.party.1564.0",
    "UNFCCC.party.1483.0",
    "UNFCCC.party.719.0",
    "UNFCCC.party.603.0",
    "UNFCCC.party.64.0",
    "UNFCCC.party.119.0",
    "UNFCCC.party.857.0",
    "UNFCCC.party.1463.0",
    "UNFCCC.party.856.0",
    "UNFCCC.party.588.0",
    "UNFCCC.non-party.280.0",
    "UNFCCC.party.29.0",
    "UNFCCC.party.176.0",
    "UNFCCC.non-party.46.0",
    "UNFCCC.party.403.0",
    "UNFCCC.party.830.0",
    "UNFCCC.non-party.297.0",
    "UNFCCC.party.839.0",
    "UNFCCC.party.68.0",
    "UNFCCC.party.581.0",
]

missing_docs_ids += extra_ids

len(missing_docs_ids)

148

In [73]:
# Import scraper CSV and clean up
metadata_df = pd.read_csv(
    "/Users/kalyan/Documents/CPR/unfccc-global-stocktake-documents/CPR_UNFCCC_MASTER.csv"
)
metadata_df[["Author", "Submission Type"]] = metadata_df[
    ["Author", "Submission Type"]
].applymap(lambda i: i.split(","))

In [74]:
missing_docs_metadata = metadata_df[
    metadata_df["CPR Document ID"].isin(missing_docs_ids)
]

ids_failed_to_find = set(missing_docs_ids) - set(
    missing_docs_metadata["CPR Document ID"].tolist()
)
print(f"IDs not found in scraper CSV: {list(ids_failed_to_find)}")

IDs not found in scraper CSV: []


In [75]:
cols_to_export = [
    # 'Category',
    "Submission Type",
    # 'Family Name',
    "Document Title",
    "Documents",
    "Author",
    "Author Type",
    # 'Geography',
    # 'Geography ISO',
    "Date",
    # 'Document Role',
    # 'Document Variant',
    # 'Language',
    # 'CPR Collection ID',
    # 'CPR Document ID',
    # 'CPR Family ID',
    # 'CPR Family Slug',
    # 'CPR Document Slug',
    # 'CPR Document Status',
    # 'Download URL'
]

main_table_for_display = missing_docs_metadata[cols_to_export].rename(
    columns={"Documents": "Source URL"}
)
main_table_for_display[["Submission Type", "Author"]] = main_table_for_display[
    ["Submission Type", "Author"]
].applymap(lambda i: ", ".join(i))
main_table_for_display = main_table_for_display.sort_values(
    ["Author Type", "Author", "Submission Type"]
)
main_table_for_display = main_table_for_display[
    ["Author Type", "Author", "Submission Type", "Document Title", "Source URL", "Date"]
]

parties_table = main_table_for_display[main_table_for_display["Author Type"] == "Party"]
non_parties_table = main_table_for_display[
    main_table_for_display["Author Type"] == "Non-Party"
]

# with pd.option_context("display.max_columns", None, "display.max_rows", None):
#     display(main_table_for_display)

## Generate markdown

In [76]:
markdown_table_parties = parties_table.to_markdown(index=False)
markdown_table_non_parties = non_parties_table.to_markdown(index=False)

In [79]:
MARKDOWN = f"""
# Missing Documents

We have yet to process {len(missing_docs_ids)} documents of the {len(metadata_df)} documents ({round(len(missing_docs_ids)/len(metadata_df)*100, 1)}%) due to formatting and technical issues. We are working on resolving this issue as soon as possible.

The documents and summary statistics are available on this page.

## Full list of missing documents

### From Parties

{markdown_table_parties}

### From non-Parties

{markdown_table_non_parties}

"""

## Export markdown

In [80]:
MARKDOWN_EXPORT_PATH = Path("../docs/UNPROCESSED_DOCUMENTS.md")
MARKDOWN_EXPORT_PATH.write_text(MARKDOWN)

54562